# Modelling the Word Game Wordle

The following notebook was created by Ken Pierce. The intention here give some practice in identifying and including pre-conditions, post-conditions, and data invariants using assertions in Python. These are fundamental concepts in creation of formal models, and the use of Python assertions allows us to explore them in a familiar setting.

## Wordle

Wordle is a word game created by Josh Wardle (https://twitter.com/powerlanguish); it is now owned and published by the New York Times: https://www.nytimes.com/games/wordle/index.html. The instructions for Wordle are as follows:

* Guess the WORDLE in 6 tries.
* Each guess must be a valid 5 letter word. 
* The color of the tiles will change to show how close your guess was to the word.
* A green letter shows it is in the correct position.
* A yellow letter shows it is in the word but in the wrong position.
* A grey letter shows that it is not in the word in any position.

## Building Models

There is no right or wrong way to construct a programme (or formal specification) from natural language specifications. We need to consider the purpose, which guides the abstraction decisions. The steps however can be broadly broken down as:

1. Analyse the functional behaviour from the requirements
2. Extract a list of possible data types (often from nouns) and functions (often from actions)
3. Create a dictionary by giving explanations to items in the list
4. Sketch out data types
5. Sketch out functions
6. Refine and add restrictions
7. Review and refine

The following will guide us through these steps to define a Wordle game.

## Functional Behaviour and Data Types

From the description above, we can identify the key elements of the instructions to understand the functionality and suggest data types:

* There’s a secret **wordle**
    - It has **five** letters (we can infer this though it is not explicit)
    - The player loses after **six** tries
* The player can make a **guess**
    - Guess must be **five** letters
    - Guess must be **valid** (= real word?)
* The game should output for each letter in a guess:
    - If the letter is in the **correct position**
    - If the letter in the **wrong position** (but in the word)
    - If the letter is **not in the word**

# The Programme

## Setup

Here, we simply include some imports for the later, as well as download a list of valid 5-letter words. Note, if the download  does not work, you can simply define a list of custom list of 5-letter strings, i.e. `WORDS = ["HELLO", ...]`.

In [2]:
from dataclasses import dataclass   # for defining dataclasses
from enum import Enum               # for defining enumerations
import typing                       # for type hinting
import random                       # for selecting a random answer
from typing import List             # for defining custom typed lists
from urllib.request import urlopen  # to load web data
# import ssl

# grab a list of words
url = 'https://raw.githubusercontent.com/tabatkins/wordle-list/main/words'
# context = ssl._create_unverified_context()
WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url).readlines()]



## Data Types and Contants

Next, we can define some simple data types, and constants that we will need to use, based on the above analysis. Here we define:

* The length of a valid word.
* The maximum number of guesses.
* A `Word` type as an alias of `str`
* An enumeration of the three types of clue (i.e. colours)
* An enumeration for the game state, either in play, won, or lost.

In [3]:
# constants
WORD_LENGTH = 5 
MAX_GUESSES = 6

# define Word as an alias for string
Word = str

# type enumerating the three possible clue colours
Clue = Enum('Clue', ['GREEN', 'YELLOW', 'GREY'])

# type enumerating the state of the game
Gamestate =  Enum('Gamestate', ['WON', 'LOST', 'PLAYING'])

## Guess

We can now introduce a `Guess` type, which represents a guess after it is processed by the game (i.e. one line of the output). We use the `dataclass' decorator and properties to simplify definition. This includes the word that was guessed, and the clues.

1. There are two invariants to add to the `setter` functions; what are these? Complete the assertions and include an appropriate message for when they fail. These should restrict the values that `word` and `clues` can take. 
2. Create some test values for `Guess` to demonstrate the invariants, i.e.  `g1 = Guess("HELLO", [Clue.YELLOW, ...])`.

In [4]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        # invariant
        assert len(word) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.word) <> {WORD_LENGTH}"  
        self._word = word

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        # invariant
        assert len(clues) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.clues) <> {WORD_LENGTH}"              
        self._clues = clues

    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"       

g1 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
#g2 = Guess("HELL", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
#g3 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN]) 

# Auxiliary Functions

There are multiple ways to specify the game. The suggestion here is to have a main class `Game` that includes the state and top-level functions including `make_guess`. To help define those, we define two auxiliary functions:

* `check_letter`: Compute clue given a character, its position, and word
* `check_guess`: Compute clues given a word and the answer

## check_letter 

This function computes the clue (colour) for a given letter, given a word and the index of the letter in the word. We will start with a naïve implementation:

* If it's in the right place, return green (i.e. `Clue.GREEN`)
* If it’s not there, return grey
* Otherwise, return yellow.

Complete the function in the following way:

3. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
4. Complete the implementation based on the sketch above.
5. Include some test cases to check your implementation, e.g. `print(check_letter("S", 0, "STOUT"))`.

In [5]:
def check_letter(letter: str, index: int, word: Word) -> Clue:
    """
    Given a letter and an index, computes the colour of the blue
    based on the word.
    """
    # pre-condition
    assert 0 <= index < WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_letter failed."
           
    if word[index] == letter: return Clue.GREEN  
    elif letter not in word: return Clue.GREY
    else: return Clue.YELLOW

print(check_letter("S", 0, "STOUT"))
print(check_letter("S", 3, "STOUT"))
print(check_letter("Z", 0, "STOUT"))

Clue.GREEN
Clue.YELLOW
Clue.GREY


## Check Guess

Given a word and the correct answer, this function computes the list of clues. This can be achieved by declaring a local variable, and for each letter in the guess, use `check_letter`; we can use `enumerate(guess)` to get the index.

6. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
7. Complete the implementation based on the sketch above.
8. Include some test cases to check your implementation, e.g. `print(check_guess("STAND", "STOUT"))`.

In [6]:

def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    # pre-condition
    assert len(word) == WORD_LENGTH and \
           len(guess) == WORD_LENGTH, "pre-check_guess failed"
           
    clues = []
    for i,letter in enumerate(guess):
        clue = check_letter(guess[i], i ,word)
        clues.append(clue)
    return clues

print(check_guess("STAND", "STOUT"))

[<Clue.GREEN: 1>, <Clue.GREEN: 1>, <Clue.GREY: 3>, <Clue.GREY: 3>, <Clue.YELLOW: 2>]


# Game State and Top-level Functionality

Here we define a `Game` class; the state is the current wordle (`answer`), the guesses made so far, and the state of the game (playing, won, or lost). Functionlity is included to print a message to the player about the game, and to rest the game when it is over.

## make_guess

The top-level function is `make_guesss` which takes a single word as a parameter. This function should use `check_guesss` to compute the guess and store it in `guesses`; it should then update the game state by checking if the game has been won (the word is exactly the same as the answer), or lost (the player reached the maximum number of guesses).

9. What pre-conditions should be included? These should restrict the parameters and the game state in which a guess is made (e.g. the player should not be able to guess if the game finished). Add these and an appropriate message.
10. Complete the implementation based on the sketch above, and play a game! You can alternate `game.make_guess(...)` and `game.print_state()` to play.

Note, can you spot the problem with the naïve implementation of `check_letter`?

In [7]:

class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer="STOUT"):
        """
        Constructor for game.
        """
        self.answer = answer
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        # make guesses uppercase
        word = word.upper() 

        # pre-condition
        assert self.gstate == Gamestate.PLAYING and \
               len(self.guesses) < MAX_GUESSES and \
               word in WORDS, "pre-guess failed."
               
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Prints a message to the user based on the current state of the game.
        """
        for guess in self.guesses: print(guess)
        if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
        else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING
   
game = Game()
game.print_state()
game.make_guess("stand")
game.print_state()


Guess the wordle, you have 6 guesses remaining.
STAND: ['S: GREEN', 'T: GREEN', 'A: GREY', 'N: GREY', 'D: GREY']
Guess the wordle, you have 5 guesses remaining.


# CSC2034 Wordle Project - Mark Hudson

## Task 2 - Hint Function

In [8]:
# type string for hint
Hint = str

def hint(word: Word, guesses: List[Guess]) -> Hint:
  """
  Return random letter from the word that has not been guessed yet.
  """
  # pre-condition
  assert len(word) == WORD_LENGTH and \
         all(len(guess.word) == WORD_LENGTH for guess in guesses), "pre-hint failed"
  
  guessed_letters = set()
  for guess in guesses:
      for letter in guess.word:
          guessed_letters.add(letter)

  unguessed_letters = [letter for letter in word if letter not in guessed_letters]
  
  if unguessed_letters:
      hint = random.choice(unguessed_letters)
      assert len(hint) == 1 and hint.isalpha() and hint.isupper(), "post-hint failed"
      return hint
  else:
      return None
  
class Game:
  answer: Word
  guesses: List[Guess]
  gstate: Gamestate

  def __init__(self, answer="BELLS"):
      """
      Constructor for game.
      """
      self.answer = answer
      self.guesses = []
      self.gstate = Gamestate.PLAYING

  def make_guess(self, word: Word):
    """
    Make a guess at the wordle.
    """
    # make guesses uppercase
    word = word.upper() 

    # pre-condition
    assert self.gstate == Gamestate.PLAYING and \
            len(self.guesses) < MAX_GUESSES and \
            word in WORDS, "pre-guess failed."
            
    self.guesses.append(Guess(word, check_guess(self.answer, word)))
    if word == self.answer: self.gstate = Gamestate.WON
    elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST
      
  def get_hint(self) -> Hint:
    """
    Get a hint for the current game.
    """
    # pre-condition
    assert self.gstate == Gamestate.PLAYING, "pre-get_hint failed."
    return hint(self.answer, self.guesses)

  def print_state(self):
    """
    Prints a message to the user based on the current state of the game.
    """
    for guess in self.guesses: print(guess)
    if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
    elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
    else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

  def game_over(self) -> bool:
    """
    Yields true if the game is over (won or lost), false otherwise.
    """
    return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 
  
  def reset(self):
    """
    Reset the game by picking a new word, clearing the guess, and
    setting the state back to playing.
    """
    # pre-condition
    assert self.game_over(), "Cannot reset, game in play"
    self.answer = random.choice(WORDS)
    self.guesses = []
    self.gstate = Gamestate.PLAYING

## Task 3 - Improving Check Guess/Check Letter

In [9]:
from collections import Counter

def check_guess(word: Word, guess: Word) -> List[Clue]:
  """
  Given the answer and a guess, compute the list of 
  clues corresponding to each letter.
  """
  # pre-condition
  assert len(word) == WORD_LENGTH and \
          len(guess) == WORD_LENGTH, "pre-check_guess failed"
           
  clues = [Clue.GREY] * WORD_LENGTH
  
  # letter_counts = {}
  # for letter in word:
  #     if letter in letter_counts:
  #         letter_counts[letter] += 1
  #     else:
  #         letter_counts[letter] = 1

  letter_counts = Counter(word)

  for i, letter in enumerate(guess):
      if letter == word[i]:
          clues[i] = Clue.GREEN
          letter_counts[letter] -= 1

  for i, letter in enumerate(guess):
      if letter != word[i] and letter in letter_counts and letter_counts[letter] > 0:
          clues[i] = Clue.YELLOW
          letter_counts[letter] -= 1

  # post-condition
  assert len(clues) == WORD_LENGTH, "post-check_guess failed"

  return clues

## Task 4 - Hard Guess

In [10]:
class Game:
  answer: Word
  guesses: List[Guess]
  gstate: Gamestate

  def __init__(self, answer="BELLS"):
    """
    Constructor for game.
    """
    self.answer = answer
    self.guesses = []
    self.gstate = Gamestate.PLAYING

  def make_guess(self, word: Word):
    """
    Make a guess at the wordle.
    """
    # make guesses uppercase
    word = word.upper() 

    # pre-condition
    assert self.gstate == Gamestate.PLAYING and \
            len(self.guesses) < MAX_GUESSES and \
            word in WORDS, "pre-guess failed."
            
    self.guesses.append(Guess(word, check_guess(self.answer, word)))
    if word == self.answer: self.gstate = Gamestate.WON
    elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

  def hard_guess(self, word: Word):
    """
    Make a hard guess (All letters from previous guesses must be used if green or yellow).
    """

    word = word.upper()

    # pre-condition
    assert self.gstate == Gamestate.PLAYING and \
            len(self.guesses) < MAX_GUESSES and \
            word in WORDS, "pre-hard_guess failed."
    
    previous_letters = set()
    for guess in self.guesses:
        for i, letter in enumerate(guess.word):
            if guess.clues[i] in [Clue.GREEN, Clue.YELLOW]:
                previous_letters.add(letter)    
      
    print(previous_letters)
    assert all(letter in word for letter in previous_letters), "post-hard_guess failed"
    
    self.guesses.append(Guess(word, check_guess(self.answer, word)))
    if word == self.answer: self.gstate = Gamestate.WON

    elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST
      
  def get_hint(self) -> Hint:
    """
    Get a hint for the current game.
    """
    # pre-condition
    assert self.gstate == Gamestate.PLAYING, "pre-get_hint failed."
    return hint(self.answer, self.guesses)

  def print_state(self):
    """
    Prints a message to the user based on the current state of the game.
    """
    for guess in self.guesses: print(guess)
    if self.gstate == Gamestate.WON: print(f"You won! You took {len(self.guesses)} guesses.")
    elif self.gstate == Gamestate.LOST: print(f"You lost! The answer was: {self.answer}.")
    else: print(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

  def game_over(self) -> bool:
    """
    Yields true if the game is over (won or lost), false otherwise.
    """
    return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 


  def reset(self):
    """
    Reset the game by picking a new word, clearing the guess, and
    setting the state back to playing.
    """
    # pre-condition
    assert self.game_over(), "Cannot reset, game in play"
    self.answer = random.choice(WORDS)
    self.guesses = []
    self.gstate = Gamestate.PLAYING

## Task 1 - Testing


In [22]:

# """Testing check_letter"""
# try:
#   assert check_letter('S', 0, 'STOUT') == Clue.GREEN, 'Incorrect clue for letter in correct position'
#   assert check_letter('S', 1, 'STOUT') == Clue.YELLOW, 'Incorrect clue for letter in incorrect position'
#   assert check_letter('Z', 0, 'STOUT') == Clue.GREY, 'Incorrect clue for letter not in word'
#   print('All tests passed: check_letter')
# except AssertionError as e:
#   print('Error:', e)

# """Testing check_guess"""
# check_guess("STAND", "STOUT")
# try:
#   assert check_guess('STAND', 'STOUT') == [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.YELLOW], 'Incorrect clues for guess'
#   assert check_guess('STAND', 'STAND') == [Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN], 'Incorrect clues for guess'
#   print('All tests passed: check_guess')
# except AssertionError as e:
#   print('Error:', e)

# """Testing Guess class"""
# try:
#   g = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
#   assert g.word == "HELLO", 'Incorrect word'
#   # assert g.clues == [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW], 'Incorrect clues'
#   print('All tests passed: Guess')
# except AssertionError as e:
#   print('Error:', e)

# """Testing Game class"""

import unittest

class TestWordle(unittest.TestCase):

  def test_check_letter(self):
    result = check_letter('S', 0, 'STOUT')
    self.assertEqual(result, Clue.GREEN)
    print(f"test_check_letter: Expected Clue.GREEN, got {result} for check_letter('S', 0, 'STOUT')")
    
    result = check_letter('S', 1, 'STOUT')
    self.assertEqual(result, Clue.YELLOW)
    print(f"test_check_letter: Expected Clue.YELLOW, got {result} for check_letter('S', 1, 'STOUT')")
    
    result = check_letter('Z', 0, 'STOUT')
    self.assertEqual(result, Clue.GREY)
    print(f"test_check_letter: Expected Clue.GREY, got {result} for check_letter('Z', 0, 'STOUT')")

  def test_check_guess(self):
    result = check_guess('STAND', 'STOUT')
    self.assertEqual(result, [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.YELLOW])
    print(f"test_check_guess: Expected [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.YELLOW], got {result} for check_guess('STAND', 'STOUT')")
    
    result = check_guess('STAND', 'STAND')
    self.assertEqual(result, [Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN])
    print(f"test_check_guess: Expected [Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN, Clue.GREEN], got {result} for check_guess('STAND', 'STAND')")

  def test_guess(self):
    g = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW])
    self.assertEqual(g.word, "HELLO")
    print(f"test_guess: Expected 'HELLO', got {g.word} for Guess word")

  def test_game(self):
    game = Game()
    game.make_guess("stand")
    self.assertEqual(game.gstate, Gamestate.PLAYING)
    print(f"test_game: Expected Gamestate.PLAYING, got {game.gstate} after first guess")
    
    game.make_guess("STAND")
    self.assertEqual(game.gstate, Gamestate.WON)
    print(f"test_game: Expected Gamestate.WON, got {game.gstate} after second guess")
    
    game.reset()
    self.assertEqual(game.gstate, Gamestate.PLAYING)
    print(f"test_game: Expected Gamestate.PLAYING, got {game.gstate} after reset")

  def test_hard_guess(self):
    game = Game()
    game.make_guess("stand")
    game.hard_guess("STAND")
    self.assertEqual(game.gstate, Gamestate.WON)
    print(f"test_hard_guess: Expected Gamestate.WON, got {game.gstate} after hard guess")

unittest.main(argv=[''], verbosity=2, exit=False)

test_check_guess (__main__.TestWordle) ... FAIL
test_check_letter (__main__.TestWordle) ... ok
test_game (__main__.TestWordle) ... FAIL
test_guess (__main__.TestWordle) ... ok
test_hard_guess (__main__.TestWordle) ... 

test_check_letter: Expected Clue.GREEN, got Clue.GREEN for check_letter('S', 0, 'STOUT')
test_check_letter: Expected Clue.YELLOW, got Clue.YELLOW for check_letter('S', 1, 'STOUT')
test_check_letter: Expected Clue.GREY, got Clue.GREY for check_letter('Z', 0, 'STOUT')
test_game: Expected Gamestate.PLAYING, got Gamestate.PLAYING after first guess
test_guess: Expected 'HELLO', got HELLO for Guess word
{'S'}


FAIL

FAIL: test_check_guess (__main__.TestWordle)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/z4/5zc07t3s1m98dq32zq76cv0m0000gn/T/ipykernel_13959/2085056671.py", line 49, in test_check_guess
    self.assertEqual(result, [Clue.GREEN, Clue.GREEN, Clue.GREY, Clue.GREY, Clue.YELLOW])
AssertionError: Lists differ: [<Clu[16 chars]ue.GREEN: 1>, <Clue.GREY: 3>, <Clue.GREY: 3>, <Clue.GREY: 3>] != [<Clu[16 chars]ue.GREEN: 1>, <Clue.GREY: 3>, <Clue.GREY: 3>, <Clue.YELLOW: 2>]

First differing element 4:
<Clue.GREY: 3>
<Clue.YELLOW: 2>

  [<Clue.GREEN: 1>,
   <Clue.GREEN: 1>,
   <Clue.GREY: 3>,
   <Clue.GREY: 3>,
-  <Clue.GREY: 3>]
+  <Clue.YELLOW: 2>]

FAIL: test_game (__main__.TestWordle)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/z4/5zc07t3s1m98dq32zq76cv0m0000gn/T/ipykernel_13959/2085056671.py", line 68, in test_game
    self.as